In [52]:
import os
import string
import numpy as np
import pandas as pd
import nltk
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM, Dropout, Flatten, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [3]:
df = pd.read_excel(r"C:\Users\Glenn\personal\Bookmark-Et\bw_reviews.xlsx", "reviews")

In [4]:
df.head()

,Content,Score
0,Easy to order but despatch and delivery to Can...,4
1,Quick process of orders. Very good prices comp...,5
2,So far it's been quick and easy to order. Wil...,5
3,"Very easy to navigate site, easy to order from...",5
4,I am upgrading my original review to 4 stars. ...,4


In [42]:
df['Score'].value_counts()

Score
5    8417
4     691
1     449
3     284
2     159
Name: count, dtype: int64

In [43]:
x = df['Content']
y = df['Score']

In [44]:
score_list = []
for i in y:
    score = i - 1
    score_list.append(score)
score_list = np.asarray(score_list)
score_list = to_categorical(score_list, num_classes=5)
print(score_list)

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [45]:
punct = string.punctuation
punct

stopwords = list(STOP_WORDS)

In [46]:
nlp = spacy.load('en_core_web_sm')
def text_cleaning(content):
    doc = nlp(content)
    
    tokens = []
    for token in doc:
        if token.lemma != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
        
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [47]:
tfidf = TfidfVectorizer(tokenizer=text_cleaning)

In [48]:
classifier = LinearSVC()

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [50]:
clf = Pipeline([('tfidf', tfidf), ('clf',classifier)])

In [51]:
clf.fit(x_train, y_train)

C:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_cleaning at 0x000001F3DB35BEC0>)),
                ('clf', LinearSVC())])

In [53]:
y_pred = clf.predict(x_test)

In [54]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.60      0.61      0.60        87
           2       0.10      0.03      0.05        33
           3       0.35      0.19      0.24        59
           4       0.38      0.11      0.17       143
           5       0.90      0.99      0.94      1678

    accuracy                           0.87      2000
   macro avg       0.47      0.38      0.40      2000
weighted avg       0.82      0.87      0.84      2000



In [55]:
accuracy_score(y_test, y_pred)

0.867

In [59]:
#Rating 5
clf.predict(["Good service, great prices, smooth transaction and great delivery time"])

array([5], dtype=int64)

In [60]:
#Rating 4
clf.predict(["Could be better, delivery time took a little longer but product was in good condition"])

array([4], dtype=int64)

In [58]:
#Rating 3
clf.predict(["Nothing too great about their services. I have seen other sites with similar prices and delivery time. Overall not too bad"])

array([5], dtype=int64)

In [61]:
#Rating 2
clf.predict(["Product arrived in a pretty bad condition, tried asking for a new product or a refund but got no response instead"])

array([1], dtype=int64)

In [62]:
#Rating 1
clf.predict(["Service was horrible, paid for an item and have been waiting for months, did not get any response from them"])

array([1], dtype=int64)